In [12]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [13]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dataa = requests.get(url).text
soup = BeautifulSoup(dataa, 'lxml')

In [14]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
 
 

In [15]:
df.head(103)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [16]:
df.shape

(103, 3)

In [17]:
df.to_csv(r"C:\Users\oluda\Study Centre\Data Science\toronto_data.csv", index=False)
print("Saved!")

Saved!


In [18]:
df = pd.read_csv(r"C:\Users\oluda\Study Centre\Data Science\toronto_data.csv")
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**Geocoder not working for me, so let's load the csv file that has the geographical coordinates of each postal code**

In [20]:
toronto_geospatial = pd.read_csv(r"C:\Users\oluda\Study Centre\Data Science\Dataset\toronto_geospatial.csv")
toronto_geospatial

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


**Merging the Geospatial with the Toronto Borough Data to Form a new DataFrame**

In [21]:
toronto_borough = df.join(toronto_geospatial.set_index('Postal Code'), on='PostalCode')

In [22]:
toronto_borough.shape

(103, 5)

In [23]:
toronto_borough

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [24]:
df.to_csv(r"C:\Users\oluda\Study Centre\Data Science\toronto_borough.csv", index=False)
print("Saved!")

Saved!


In [25]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
!pip install geopy==2.1.0
from geopy.geocoders import Nominatim
!pip install folium==0.5.0
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

print("Libraries Imported!")

Processing c:\users\oluda\appdata\local\pip\cache\wheels\ef\4c\4a\17fd3d7fb7b6243d5a7a8d165870cd5c6ad2ec4c0582f039e4\folium-0.5.0-py3-none-any.whl
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1
    Uninstalling folium-0.12.1:
      Successfully uninstalled folium-0.12.1
Libraries Imported!


In [26]:
pip install folium

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: six in c:\users\oluda\anaconda3\lib\site-packages (from folium) (1.15.0)



# Getting Venues Data

## Coordinates for Toronto

In [28]:
# Create a Nominatim object for geolocation
geolocator = Nominatim(user_agent="ny_explorer")

# GEt latitue and longitude values for Toronto
location = geolocator.geocode('Toronto')
lat = location.latitude
lon = location.longitude

# Display latitude and longitude values for Toronto
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, lon))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create Map for Toronto

In [30]:
# Create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add markers to map
for lat, lon, borough, neighborhood in zip(canada_df['Latitude'], canada_df['Longitude'], canada_df['Borough'], canada_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], 
                        radius=5, 
                        popup=label, 
                        color='blue', 
                        fill=True, 
                        fill_color='#3186cc', 
                        fill_opacity=0.7, 
                        parse_html=False).add_to(toronto_map)  

# Display map for Toronto   
toronto_map

NameError: name 'canada_df' is not defined

In [31]:
# Your Foursqaure API Client ID
CLIENT_ID = 'ZLPAURQKU5T2ENSTGHMMLTRTSEHOR2PK3NMDFVY3C4KXYQY4'

# Your Foursquare API CLient secret
CLIENT_SECRET = 'ZI0U1EYG25XOY2KQCJ1XBULFDR50E44PQXMNIPJMTDWIHL4Q'

# Foursquare API version
VERSION = '20180604'

LIMIT = 100
radius = 500

# URL for getting data from Foursquare API
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, VERSION, radius, LIMIT)
print('URL: ', url)

URL:  https://api.foursquare.com/v2/venues/search?client_id=ZLPAURQKU5T2ENSTGHMMLTRTSEHOR2PK3NMDFVY3C4KXYQY4&client_secret=ZI0U1EYG25XOY2KQCJ1XBULFDR50E44PQXMNIPJMTDWIHL4Q&ll=43.6534817,-79.3839347&v=20180604&radius=500&limit=100


In [32]:
# Function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
# Send the GET request and examine the results
results = requests.get(url).json()

# Assign relevant part of JSON to venues
venues = results['response']['venues']

# Transform venues into a DataFrame
venues_df = json_normalize(venues)

# Keep only relevant columns
filtered_columns = ['name', 'categories'] + [col for col in venues_df.columns if col.startswith('location.')] + ['id']
venues_df = venues_df.loc[:, filtered_columns]

# Get venue category for each record
venues_df['categories'] = venues_df.apply(get_category_type, axis=1)

# Clean column names
venues_df.columns = [column.split('.')[-1] for column in venues_df.columns]

# Display first 10 records
venues_df.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,5b193c42598e64002ca79b96
1,Toronto City Hall,City Hall,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.,4ad4c05ef964a5208ff620e3
2,Mayor Office Toronto,Office,100 Queen St. West,43.653367,-79.384021,"[{'label': 'display', 'lat': 43.653367, 'lng':...",14,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. West, Toronto ON M5H 2N2, Canada]",NaN,4e4a878c52b11cf651b30044
3,Toronto Public Library,Library,100 Queen St W,43.652631,-79.383295,"[{'label': 'display', 'lat': 43.65263139770985...",107,M5H 2N3,CA,Toronto,ON,Canada,"[100 Queen St W (City Hall), Toronto ON M5H 2N...",City Hall,4c8938c8944e224b52e72285
4,City Hall Council Chambers,City Hall,NaN,43.651827,-79.383949,"[{'label': 'display', 'lat': 43.65182710471462...",184,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4c093ee0340720a153728493
5,Kew Gardens Playground,Playground,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN,4c41d47cd691c9b6fa4c8d0a
6,City Hall Podium Green Roof,Garden,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,4c0121fd9a950f47fa9208c6
7,Cafe On The Square,Restaurant,100 Queen St W,43.652633,-79.383361,"[{'label': 'display', 'lat': 43.65263298456729...",105,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,4b06ef4cf964a52072f322e3
8,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",130,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Toronto ON M5G 2C9, Canada]",NaN,50885719498ea7b5aab3a74c
9,CIBC,Office,Bell Trinity Square,43.653345,-79.382068,"[{'label': 'display', 'lat': 43.65334506217502...",151,NaN,CA,Toronto,ON,Canada,"[Bell Trinity Square (Bay St.), Toronto ON, Ca...",Bay St.,4b0ab365f964a5209b2623e3


In [34]:
# Function to get nearby venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # URL to get data about venues from Foursquare API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, radius, LIMIT)   
        
        # Get data through get request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Add results to list of all venues
        venues_list.append([(name, lat, lon, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    # Create DataFrame for nearby venues
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    # Return DataFrame of nearby venues
    return(nearby_venues)

In [35]:
# Get nearby venues for Toronto
toronto_venues = getNearbyVenues(names=canada_df['Neighborhood'], latitudes=canada_df['Latitude'], longitudes=canada_df['Longitude'])

# Print number of nearby venues retuirned
print('Foursquare retured {} nearby venues for Toronto.'.format(toronto_venues.shape[0]))

# Display first 10 records for nearby venues for Toronto
toronto_venues.head(10)

NameError: name 'canada_df' is not defined

In [36]:
# Print number of unique venue categories
print('There are {} uniques categories of venues.'.format(len(toronto_venues['Venue Category'].unique())))

NameError: name 'toronto_venues' is not defined

# Data Processing

## One-hot Encoding

In [38]:
# Perform one-hot encoding for Toronto venues DataFrame
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to DataFrame
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Display first 10 records
toronto_onehot.head(100)

NameError: name 'toronto_venues' is not defined

In [39]:
# Group all records by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

# Display first 10 records
toronto_grouped.head(10)

NameError: name 'toronto_onehot' is not defined

## Exploring & Clustering

In [40]:
# Funtion to get the most comkmong venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
# Get top 5 venues for each neighborhodd
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new DataFrame for neighborhoods and most common venues
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# Display first 10 records
neighborhoods_venues_sorted.head(10)

NameError: name 'toronto_grouped' is not defined